In [1]:
from sklearn import *

import numpy as np
import pandas as pd
import sklearn as sk 
import sklearn.model_selection
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import AdaBoostClassifier
# from sklearn.ensemble import GradientBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
# from sklearn.ensemble import GradientBoostingRegressor

import time

#  # For Alex's machine
# df = pd.read_csv("Data/Train.csv")

# For Gokul's machine
df = pd.read_csv("./train.csv")

#extract the review summary columm
#print(df["summary"], df["overall"])

#array of all the summaries

og_summaries = df['summary']
summaries = df["summary"]

#array of the overalls
overall = df["overall"]

#array of product nums
products = df["amazon-id"]

og_amazon_reviews = df['reviewText']
amazon_reviews = df['reviewText']

#array of artists 
artist_arr = df['artist']

In [2]:
#extracting IMDB movie reviews

# For Gokul's machine
# moviesdf = pd.read_csv("./labeledTrainData.tsv", delimiter = "\t")
# moviesdf = pd.read_csv('Data/imdb_data.csv')
# moviesdf.head()

In [3]:
# sentiments = [int(sentiment=='positive') for sentiment in moviesdf['sentiment']]

In [4]:
# Using ratings associated with reviews as indicators of sentiment
review_sentiments = [int(overall[i] >= 5) for i in range(len(overall))]

In [5]:
for i in range(len(amazon_reviews)):
    if type(amazon_reviews[i])!=str:
        amazon_reviews.pop(i)
        review_sentiments.pop(i)
print(set([type(r) for r in amazon_reviews]))

{<class 'str'>}


In [6]:
# reviews = moviesdf["review"]
#sentiment = moviesdf["sentiment"]

#create train test split
x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(amazon_reviews, 
                                                                       review_sentiments, 
                                                                       random_state=1)

In [7]:
#create model
vect = HashingVectorizer()

x_train = vect.fit_transform(x_train)
x_test = vect.transform(x_test)

# We can experiment with different numbers of max iterations
# 100 iters is default
# iters = [1000]

start = time.time()
review_clf = LogisticRegression(max_iter=500)
review_clf.fit(x_train, y_train)
end = time.time()
print("Test accuracy: {}".format(100*review_clf.score(x_test, y_test)))
print("Time taken: {:.2f} seconds".format(end-start))

/Users/gokulsrin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Test accuracy: 80.91740476704831
Time taken: 4.71 seconds


In [8]:
summary_sentiments = [int(overall[i] >= 5) for i in range(len(overall))]
for i in range(len(summaries)):
    if type(summaries[i])!=str:
        summaries.pop(i)
        summary_sentiments.pop(i)
print(set([type(s) for s in summaries]))

{<class 'str'>}


In [9]:
x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(summaries, 
                                                                       summary_sentiments, 
                                                                       random_state=1)

In [10]:
# create model

x_train = vect.fit_transform(x_train)
x_test = vect.transform(x_test)

# We can experiment with different numbers of max iterations
# 100 iters is default
# iters = [1000]

start = time.time()
summary_clf = LogisticRegression(max_iter=500)
summary_clf.fit(x_train, y_train)
end = time.time()
print("Test accuracy: {}".format(100*summary_clf.score(x_test, y_test)))
print("Time taken: {:.2f} seconds".format(end-start))

Test accuracy: 77.88219197810903
Time taken: 0.88 seconds


In [11]:
df = pd.read_csv("./train.csv")

In [12]:
# using this to classify shit

summary_data = vect.transform(df['summary'].values.astype('U'))
review_data = vect.transform(df['reviewText'].values.astype('U'))

In [13]:
print(review_data.shape)

(111098, 1048576)


In [14]:
summary_predictions = summary_clf.predict(summary_data)
review_predictions = review_clf.predict(review_data)

In [15]:
#accuracy analysis

def getAccuracy(texts, preds):
    total = len(texts)
    t = 0
    for i in range(total):
        if (overall[i] >= 4 and preds[i] == 1) or (overall[i] < 4 and preds[i] == 0):
            t+=1
    return t/total

In [16]:
print("Accuracy of summary sentiment classifier on Amazon summaries: {:.2f}".format(getAccuracy(summaries, summary_predictions)))
print("Accuracy of review sentiment classifier on Amazon reviews: {:.2f}".format(getAccuracy(amazon_reviews, review_predictions)))

Accuracy of summary sentiment classifier on Amazon summaries: 0.87
Accuracy of review sentiment classifier on Amazon reviews: 0.86


In [17]:
# We'll now train three classifiers:
# 1. Using just avg. summary sentiment as a feature
# 2. Using just avg. review sentiment as a feature
# 3. Using both (1) and (2) as features

In [18]:
targets = {}
for i in range(len(products)):
    if products[i] not in targets:
        targets[products[i]] = [overall[i]]
    else:
        targets[products[i]].append(overall[i])

In [19]:
targets = [int(np.average(targets[prod]) > 4.5) for prod in targets]

In [20]:
print(targets[:10])

[0, 1, 1, 0, 1, 1, 1, 1, 0, 0]


In [21]:
review_predictions, summary_predictions = list(review_predictions), list(summary_predictions)

In [22]:
amz_review_sentiments, amz_summary_sentiments = {}, {}
for i in range(len(products)):
    if products[i] not in amz_review_sentiments:
        amz_review_sentiments[products[i]] = [review_predictions[i]]
        amz_summary_sentiments[products[i]] = [summary_predictions[i]]
    else:
        amz_review_sentiments[products[i]].append(review_predictions[i])
        amz_summary_sentiments[products[i]].append(summary_predictions[i])

In [23]:
avg_review_sentiments = [np.average(amz_review_sentiments[prod]) for prod in amz_review_sentiments]

In [24]:
avg_summary_sentiments = [np.average(amz_summary_sentiments[prod]) for prod in amz_summary_sentiments]

In [25]:
avg_review_sentiments = [[score] for score in avg_review_sentiments]
avg_summary_sentiments = [[score] for score in avg_summary_sentiments]

In [26]:
X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(avg_summary_sentiments, targets, random_state=1)


#train model
summary_only_clf = LogisticRegression()
summary_only_clf.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [27]:
print("Test accuracy using summary sentiment only: {:.2f}".format(100*summary_only_clf.score(X_test, Y_test)))

Test accuracy using summary sentiment only: 73.18


In [28]:
X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(avg_review_sentiments, targets, random_state=1)


#train model
review_only_clf = LogisticRegression()
review_only_clf.fit(X_train, Y_train)

/Users/gokulsrin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
print("Test accuracy using review sentiment only: {:.2f}".format(100*review_only_clf.score(X_test, Y_test)))

Test accuracy using review sentiment only: 74.43


In [ ]:
#here we develop the artist feature with just summary predictions
artist = {}
for i in range(len(summary_predictions)):
    if artist.get(artist_arr[i]) == None:
        artist[artist_arr[i]] = [summary_predictions]
    else:
        artist.get(artist_arr[i]).append(summary_predictions)
for a in artist:
    artist[a] = np.average(artist.get(a))
    print('aritst:',artist,"; score:",artist.get(a))
print("hello")

In [ ]:
X = np.zeros((len(avg_review_sentiments), 2))

In [ ]:
for i in range(X.shape[0]):
    X[i] = (avg_review_sentiments[i][0], avg_summary_sentiments[i][0])

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=1)

In [ ]:
targets = [[t] for t in targets]

In [ ]:
targets = np.asarray(targets)

In [ ]:
for train_idx, test_idx in kf.split(X):
    X_train, X_test = X[train_idx], X[test_idx]

for train_idx, test_idx in kf.split(targets):
    y_train, y_test = targets[train_idx], targets[test_idx]

In [ ]:
print(len(X_train), len(y_train))

In [ ]:
print(len([s for s in targets if s[0]==1]))
print(len([s for s in targets if s[0]==0]))

In [ ]:
print(6181/4362)

In [ ]:

X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(X, targets, random_state=1)


#train model
review_and_summary_clf = SVC(kernel='linear', class_weight = {0: 1.417, 1:1})
# review_and_summary_clf = AdaBoostClassifier(SVC(probability=True,kernel='linear', class_weight = {0: 1.417, 1:1}),n_estimators=10, random_state=1)
review_and_summary_clf.fit(X_train, Y_train)

In [ ]:
print("Test accuracy using review sentiment and summary sentiment: {:.2f}".format(100*review_and_summary_clf.score(X_test, Y_test)))

In [ ]:
print("Weighted F1 using review sentiment and summary sentiment: {:.2f}".format(100*f1_score(Y_test, 
             review_and_summary_clf.predict(X_test), 
             average='weighted')))

In [ ]:
sales_ranks = {}
for i in range(len(products)):
    if products[i] not in sales_ranks:
        sales_ranks[products[i]] = [df['salesRank'][i]]
    else:
        sales_ranks[products[i]].append(df['salesRank'][i])

In [ ]:
avg_sales_ranks = [[np.average(sales_ranks[prod])] for prod in sales_ranks]

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
scaler = StandardScaler() # 73.6 weighted F1
# scaler = MinMaxScaler() # 73.6 weighted F1
# scaler = MaxAbsScaler() # 73.6 weighted F1
# scaler = RobustScaler() # 73.6 weighted F1
# scaler = Normalizer() # 73.48 weighted F1
# scaler = QuantileTransformer() # 73.48 weighted F1
# scaler = PowerTransformer() # 73.48 weighted F1

In [ ]:
scaler.fit(avg_sales_ranks)
avg_sales_ranks = scaler.transform(avg_sales_ranks)

In [ ]:
X = np.zeros((len(avg_review_sentiments), 3))

In [ ]:
for i in range(X.shape[0]):
    X[i] = (avg_review_sentiments[i][0], avg_summary_sentiments[i][0], avg_sales_ranks[i][0])

In [ ]:
for train_idx, test_idx in kf.split(X):
    X_train, X_test = X[train_idx], X[test_idx]

for train_idx, test_idx in kf.split(targets):
    y_train, y_test = targets[train_idx], targets[test_idx]

In [ ]:
review_summary_sales_clf = SVC(kernel='linear', class_weight = {0: 1.417, 1:1})
review_summary_sales_clf.fit(X_train, y_train)

In [ ]:
print("Test accuracy using review sentiment, summary sentiment, and sales rank: {:.2f}".format(100*review_summary_sales_clf.score(X_test, y_test)))

In [ ]:
print("Weighted F1 using review sentiment, summary sentiment, and sales rank: {:.2f}".format(100*f1_score(y_test, 
             review_summary_sales_clf.predict(X_test), 
             average='weighted')))

In [ ]:
# # to get better F1 score
# yhat = review_summary_sales_clf.predict_proba(X_test)
# probs = yhat[:,1]
# thresholds = np.arrange(0,1,0.001)
# def to_labels(pos_prob, threshold):
#     return (pos_prob >= threshold).astype('int')
# scores = [f1_score(y_test, to_labels(probs, t), average ='weighted') for t in thresholds]
# argmax = np.argmax(scores)
# print('Best threshold: {:.2f}, Best weighted {:.2f}'.format(thresholds[argmax], scores[argmax]))

In [ ]:
prices = {}
for i in range(len(products)):
    if products[i] not in prices:
        prices[products[i]] = [df['price'][i]]
    else:
        prices[products[i]].append(df['price'][i])

In [ ]:
avg_prices = [[np.average(prices[prod])] for prod in prices]

In [ ]:
scaler.fit(avg_prices)
avg_prices = scaler.transform(avg_prices)

In [ ]:
X = np.zeros((len(avg_review_sentiments), 4))
for i in range(X.shape[0]):
    X[i] = (avg_review_sentiments[i][0], avg_summary_sentiments[i][0], avg_sales_ranks[i][0], 
           avg_prices[i][0])

In [ ]:
# for train_idx, test_idx in kf.split(X):
#     X_train, X_test = X[train_idx], X[test_idx]

# for train_idx, test_idx in kf.split(targets):
#     y_train, y_test = targets[train_idx], targets[test_idx]
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, targets, random_state=1)

In [ ]:
review_summary_sales_price_clf = SVC( kernel='linear',class_weight = {0: 1.417, 1:1})
review_summary_sales_price_clf.fit(X_train, y_train)

In [ ]:
print("Test accuracy using review sentiment, summary sentiment, sales rank, and price: {:.2f}".format(100*review_summary_sales_price_clf.score(X_test, y_test)))

In [ ]:
print("Weighted F1 using review sentiment, summary sentiment, sales rank, and price: {:.2f}".format(100*f1_score(y_test, 
             review_summary_sales_price_clf.predict(X_test), 
             average='weighted')))